###import part

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/esun_ai_competition/')

In [ ]:
import sys
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as tvm
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [ ]:
!tar xvf "train.tar"

串流輸出內容已截斷至最後 5000 行。
train/55137_旺.jpg
train/55140_玉.jpg
train/16126_健.jpg
train/55355_動.jpg
train/55510_晟.jpg
train/16127_善.jpg
train/51463_安.jpg
train/51490_柏.jpg
train/16099_昭.jpg
train/16179_龍.jpg
train/57193_昌.jpg
train/16130_邦.jpg
train/18295_鎮.jpg
train/18296_娥.jpg
train/23269_啟.jpg
train/23270_設.jpg
train/29740_漁.jpg
train/16131_鑫.jpg
train/27575_公.jpg
train/27576_吳.jpg
train/61598_築.jpg
train/31373_台.jpg
train/31374_廖.jpg
train/45316_牙.jpg
train/16132_優.jpg
train/67153_昇.jpg
train/67158_毅.jpg
train/16148_沈.jpg
train/65531_沈.jpg
train/65532_鑫.jpg
train/16149_實.jpg
train/66121_薇.jpg
train/66122_僑.jpg
train/16102_芸.jpg
train/16181_技.jpg
train/21668_客.jpg
train/21669_生.jpg
train/52225_年.jpg
train/16150_泰.jpg
train/24798_油.jpg
train/24821_艾.jpg
train/24619_韻.jpg
train/16151_流.jpg
train/20398_虹.jpg
train/20399_翔.jpg
train/22188_數.jpg
train/22189_億.jpg
train/37958_耀.jpg
train/37959_產.jpg
train/49005_趙.jpg
train/49006_所.jpg
train/20071_敏.jpg
train/61513_春.jpg
train/61516_經.jpg
train/3

### Functions

In [ ]:
def readfile(path, word2idx):
  image_dir = sorted(os.listdir(path))
  x = np.zeros((len(image_dir), 64, 64, 3), dtype=np.uint8)
  y = np.zeros((len(image_dir)), dtype=np.int)
  for i, file in enumerate(tqdm(image_dir)):
    img = cv2.imread(os.path.join(path, file))
    x[i, :, :] = cv2.resize(img,(64, 64))
    label = file.split('_')[1][0]
    if label in word2idx.keys():
      y[i] = word2idx[label]
    else:
      y[i] = word2idx['isnull']
  return x, y

In [ ]:
# data augmentation
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((299, 299)),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15),
    #transforms.RandomPerspective(),
    #transforms.RandomAffine(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((299, 299)),                                    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 64, 64]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1, 1),  # [32, 64, 64]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, 1, 1),  # [32, 64, 64]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [32, 32, 32]

            nn.Conv2d(32, 64, 3, 1, 1), # [64, 32, 32]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1), # [64, 32, 32]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 16, 16]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 16, 16]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, 1, 1), # [128, 16, 16]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, 1, 1), # [128, 16, 16]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 8, 8]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 8, 8]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), # [256, 8, 8]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), # [256, 8, 8]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [256, 4, 4]

        )
        self.fc = nn.Sequential(
            nn.Linear(256*4*4, 1024),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1024, 801)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)


In [ ]:
idx2word, word2idx = {}, {}
with open('training data dic.txt') as f:
  lines = f.readlines()

idx2word[0] = 'isnull'
word2idx['isnull'] = 0
for i in range(len(lines)):
  word = lines[i].strip('\n')
  idx2word[i+1] = word
  word2idx[word] = i+1

In [ ]:
# read file
x, y = readfile('train', word2idx)

In [ ]:
length = int(len(x) / 5)
split_x, split_y = [], []
for i in range(5):
  if i == 4:
    split_x.append(x[i*length:])
    split_y.append(y[i*length:])
  else:
    split_x.append(x[i*length:(i+1)*length])
    split_y.append(y[i*length:(i+1)*length])

"""
如果要改的話改split_x跟split_y的idx
例如:
train_x = np.concatenate((np.array(split_x[1]),np.array(split_x[2]),np.array(split_x[3]),np.array(split_x[4])), axis=0)
train_y = np.concatenate((np.array(split_y[1]),np.array(split_y[2]),np.array(split_y[3]),np.array(split_y[4])), axis=0)
val_x  = np.array(split_x[0])
val_y  = np.array(split_y[0])
"""
train_x = np.concatenate((np.array(split_x[0]),np.array(split_x[1]),np.array(split_x[2]),np.array(split_x[4])), axis=0)
train_y = np.concatenate((np.array(split_y[0]),np.array(split_y[1]),np.array(split_y[2]),np.array(split_y[4])), axis=0)
val_x  = np.array(split_x[3])
val_y  = np.array(split_y[3])
print("train set = " + str(len(train_x)))
print("val set = " + str(len(val_x)))

train set = 55044
val set = 13760


### Training

In [ ]:
# batch_size = 32

# train_set = ImgDataset(train_x, train_y, train_transform)
# val_set = ImgDataset(val_x, val_y, test_transform)
# train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# model = tvm.inception_v3(init_weights=True, pretrained=False, num_classes=801).cuda()
# model.aux_logits = False
# loss = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)
# num_epoch = 10
# best_acc = 0.0

# for epoch in range(num_epoch):
#     train_acc = 0.0
#     train_loss = 0.0

#     model.train()
#     for i, data in enumerate(tqdm(train_loader)):
#         optimizer.zero_grad()
#         train_pred = model(data[0].cuda())
#         batch_loss = loss(train_pred, data[1].cuda())
#         batch_loss.backward()
#         optimizer.step()

#         train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
#         train_loss += batch_loss.item()

#     print(f"Epoch {epoch + 1} | loss = {train_loss / train_set.__len__()}, acc = {train_acc / train_set.__len__()}")
  
#     model.eval()
#     with torch.no_grad():
#       val_acc = 0.0
#       for i, data in enumerate(tqdm(val_loader)):
#           val_pred = model(data[0].cuda())
#           val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
#       if val_acc > best_acc:
#         best_acc = val_acc
#         torch.save(model, 'inception_val1.ckpt')

#       print(f"Validation | acc = {val_acc / val_set.__len__()}")



### pick out argmax

In [ ]:
model = torch.load("inception_val3.ckpt")
model.aux_logits = False
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [ ]:
val_set = ImgDataset(val_x, val_y, test_transform)
val_loader = DataLoader(val_set, batch_size=1, shuffle=False)
image_dir = sorted(os.listdir('train'))
not1_label = []
wrong_label = []
wrong_idx = []
wrong_prob = []
wrong_cnt = 0


softmax = nn.Softmax(dim = -1)
for i, data in enumerate(tqdm(val_loader)):
  with torch.no_grad():
    val_pred = model(data[0].cuda())
  
  probs = softmax(val_pred)
  probs = probs.cpu().numpy()
  idx = np.argmax(probs)

  if idx != data[1]:
    wrong_cnt +=1
    if probs[0][idx] <= 0.3 :
      not1_label.append(i)
    elif probs[0][idx] > 0.8:
      wrong_label.append(i)
      wrong_idx.append(idx)
      wrong_prob.append(probs[0][idx])
      #print("pick prob", probs[0][idx])

In [ ]:
print("pick ratio=", (len(wrong_label)+len(not1_label))/wrong_cnt)
print("wrong_cnt=", wrong_cnt)
print("pick=", len(wrong_label)+len(not1_label))
print("wrong label=", len(wrong_label))
print("not1_label=", len(not1_label))


pick ratio= 0.5699535363964894
wrong_cnt= 1937
pick= 1104
wrong label= 372
not1_label= 732


In [ ]:
for i in range(len(wrong_label)):
  wrong_label[i] += 3*length

for i in range(len(not1_label)):
  not1_label[i] += 3*length

In [ ]:
for i, file in enumerate(image_dir):
  if i in not1_label:
    tmp = file[0:-5]
    tmp = tmp + "isnull"
    print(tmp)

47159_isnull
47165_isnull
47194_isnull
47197_isnull
4719_isnull
47213_isnull
47219_isnull
47236_isnull
47241_isnull
47252_isnull
4726_isnull
47270_isnull
47277_isnull
47316_isnull
47327_isnull
47337_isnull
4733_isnull
47358_isnull
47366_isnull
47385_isnull
47388_isnull
47400_isnull
47405_isnull
47424_isnull
47447_isnull
47496_isnull
47501_isnull
47547_isnull
47569_isnull
47578_isnull
47601_isnull
4761_isnull
47624_isnull
4762_isnull
47631_isnull
47656_isnull
47664_isnull
47681_isnull
47686_isnull
47747_isnull
47825_isnull
4782_isnull
47849_isnull
47851_isnull
47857_isnull
47931_isnull
47935_isnull
47937_isnull
47979_isnull
47982_isnull
47984_isnull
47985_isnull
47992_isnull
48007_isnull
48008_isnull
48026_isnull
48059_isnull
48060_isnull
48067_isnull
48095_isnull
48113_isnull
48134_isnull
48148_isnull
48159_isnull
48173_isnull
48195_isnull
48214_isnull
48247_isnull
4824_isnull
48256_isnull
4827_isnull
48303_isnull
4835_isnull
48379_isnull
48381_isnull
48399_isnull
4842_isnull
48440_isn

In [ ]:
for i, file in enumerate(image_dir):
  if i in wrong_label:
    tmp2 = file[0:-5]
    j = wrong_label.index(i)
    tmp2 = tmp2 + idx2word[wrong_idx[j]]
    #print(file, tmp2, wrong_prob[j])
    print(tmp2)

47157_和
47174_公
47203_洲
47220_施
47226_務
47240_流
47251_高
47319_訊
47356_金
47379_所
47446_金
47488_琪
47522_園
47542_款
47549_員
47604_邦
47620_秦
4764_伶
47672_紀
47674_事
47690_卓
47757_樓
4778_萱
47793_貝
47854_意
47875_葉
47907_國
47908_告
47950_合
4795_事
47_鴻
48039_經
48092_聖
48093_凱
48169_法
48188_食
48212_煌
48217_金
48229_食
48271_數
48354_專
48369_通
48373_法
48407_來
48428_鐵
4843_羅
48514_紙
48543_邱
48595_拓
48625_艾
48626_合
48650_行
48654_工
48962_香
4897_世
48984_月
49051_信
49102_明
49173_股
49252_清
4925_資
49264_金
49306_專
49318_澤
49335_永
49345_捷
49367_理
4937_章
49389_博
49406_流
49412_所
4947_菊
4948_圓
49511_不
49578_會
49587_菁
49592_章
49626_勳
49748_三
49769_舒
49812_龍
49892_科
49904_私
49929_消
49963_器
49964_食
49970_臺
50026_祥
500_同
50119_屋
50120_膠
50128_運
50163_大
50193_客
50275_發
50278_丞
50323_機
50377_委
50407_馬
50423_兒
50436_朱
50443_朝
50486_裕
50503_音
50522_材
50526_護
50588_偉
50643_升
50665_貿
5067_會
50693_興
50730_敏
50796_代
50809_設
50816_洲
50818_保
50845_耀
50893_清
51042_師
51103_名
51157_航
51180_佩
51212_香
51250_煌
51285_員
51355_凍
51364_報